<a href="https://colab.research.google.com/github/prosperwashaya/SAR-OPTICAL_DATA_GEE/blob/main/Full_workflow_28_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google
from google.colab import auth
from google.auth import compute_engine

import ee
import folium
import geemap
import geemap.colormaps as cm
import matplotlib.pyplot as plt

In [ ]:
#authenticate with user credentials
auth.authenticate_user()

In [ ]:
auth.authenticate_user(project_id= 'awesome-treat-402108')

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
pip install tensorflow==2.15.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
# REPLACE WITH YOUR CLOUD PROJECT!
PROJECT = 'awesome-treat-402108'

# Output bucket for trained models.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'tezba7'
# OUTPUT_BUCKET = 'C:/Users/Prosper/Desktop/EE/EE_Jupyter/Bucket'

# Cloud Storage bucket with training and testing datasets.
DATA_BUCKET = 'outputbucket_tezba'

GEOMETRY = ee.FeatureCollection('users/pwashaya9/czech_shp')
GEOMETRY = GEOMETRY.geometry()


# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'Band_1'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 1

forest = ee.Image('projects/awesome-treat-402108/assets/forest_mask_full_final2')

forest_mask = forest.updateMask(forest).gt(0)


# Map.addLayer(forest)
def landuse(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    landuse_mask = forest_mask
    # Both flags should be set to zero, indicating clear conditions.
    return image.updateMask(landuse_mask)

In [ ]:
# Parameters
CLD_PRB_THRESH = 20  # Cloud probability threshold
NIR_DRK_THRESH = 0.2  # Dark NIR threshold for shadows
CLD_PRJ_DIST = 2  # Cloud shadow projection distance (km)
BUFFER = 500  # Buffer distance for cloud and shadow dilation
GEOMETRY = ee.FeatureCollection('users/pwashaya9/czech_shp').geometry()

# # Debugging Mask Function
# def debug_add_cld_shdw_mask(img):
#     is_cloud = img.select('MSK_CLASSI_OPAQUE') #.gt(CLD_PRB_THRESH) \
#                   # .And(img.select('MSK_CLASSI_OPAQUE').gte(1)) \
#                   # .Or(img.select('MSK_CLASSI_CIRRUS').gte(1))
#     scl = img.select('SCL')
#     scl_mask = scl.eq(4).Or(scl.eq(8)).Or(scl.eq(9)).Or(scl.eq(10)).Or(scl.eq(11))

#     cloud_shadow_mask = is_cloud.And(scl_mask).rename('cloudmask')
#     return img.addBands(cloud_shadow_mask).updateMask(cloud_shadow_mask.Not())

def mask_clouds(image):
    """
    Masks clouds in Sentinel-2 TOA imagery using the QA60 band.

    Args:
        image (ee.Image): Sentinel-2 TOA image.

    Returns:
        ee.Image: The cloud-masked image.
    """
    # Get the QA60 band
    qa60 = image.select('QA60')

    # Create a mask for clear pixels (non-cloudy areas have a value of 0 in QA60)
    cloud_mask = qa60.bitwiseAnd(1 << 10).eq(0).And(qa60.bitwiseAnd(1 << 11).eq(0))

    # Apply the mask to the image
    return image.updateMask(cloud_mask)

# Example usage: Load a Sentinel-2 image and apply the mask
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2024-01-01', '2024-01-31') \
    .filterBounds(ee.Geometry.Point([12.4924, 41.8902])) # Example point (Rome)

# Apply the cloud masking function to the ImageCollection
masked_collection = sentinel2.map(mask_clouds)

# Print some information about the first image in the collection
first_image = masked_collection.first()
print(first_image.getInfo())



# Function to compute indices
def compute_indices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': img.select('B8'),
            'RED': img.select('B4'),
            'BLUE': img.select('B2')
        }).rename('EVI')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    nbr = img.normalizedDifference(['B8', 'B12']).rename('NBR')
    return img.addBands([ndvi, evi, ndwi, nbr])


# Function to normalize bands
def normalize(image):
    bands = ['NDVI', 'NDWI', 'NBR']
    normalized_bands = []
    for band in bands:
        mnmx = image.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), GEOMETRY, 1000)
        mean = ee.Number(mnmx.get(band + "_mean"))
        stdDev = ee.Number(mnmx.get(band + "_stdDev"))
        normalized_band = image.select([band]).subtract(mean).divide(stdDev).add(1).multiply(64).byte()
        normalized_bands.append(normalized_band)
    normalized_image = ee.Image.cat(normalized_bands).rename(bands)
    return normalized_image


# Function to process yearly data
def process_yearly_data(year):
    start_date = f"{year}-05-01"
    end_date = f"{year}-09-30"
    collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                  .filterBounds(GEOMETRY)
                  .filterDate(start_date, end_date)
                  # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                  .map(compute_indices)
                  .map(mask_clouds)
                  .map(landuse))
    return collection.median()


# Define the start and end years
start_year = 2016
end_year = 2019

# Generate yearly mosaics
yearly_mosaics = {year: process_yearly_data(year) for year in range(start_year, end_year + 1)}

# Calculate differences for NDVI, NDWI, and NBR between consecutive years
diff_images = {}
for year in range(start_year + 1, end_year + 1):
    prev_year = year - 1
    ndvi_diff = yearly_mosaics[year].select('NDVI').subtract(yearly_mosaics[prev_year].select('NDVI'))
    ndwi_diff = yearly_mosaics[year].select('NDWI').subtract(yearly_mosaics[prev_year].select('NDWI'))
    nbr_diff = yearly_mosaics[year].select('NBR').subtract(yearly_mosaics[prev_year].select('NBR'))

    # Assign meaningful names
    diff_images[f"ndvi_diff_{year}"] = ndvi_diff
    diff_images[f"ndwi_diff_{year}"] = ndwi_diff
    diff_images[f"nbr_diff_{year}"] = nbr_diff

# Print the names of the generated difference images
for name in diff_images.keys():
    print(name)

# Visualize NDVI differences for all years
for year in range(start_year + 1, end_year + 1):
    # Calculate NDVI difference
    ndvi_diff = yearly_mosaics[year].select(f'NDVI').subtract(yearly_mosaics[year - 1].select(f'NDVI'))

    Map = geemap.Map(center=[49.8175, 15.4730], zoom=6, ee_initialize=False)

    # Add the layer to the map with a descriptive name
    Map.addLayer(ndvi_diff, {'min': -0.5, 'max': 0.5, 'palette': ['blue', 'white', 'yellow', 'red']}, f"NDVI Diff {year}")
    Map.addLayerControl()

Map


{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32632', 'crs_transform': [60, 0, 699960, 0, -60, 4700040]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 699960, 0, -10, 4700040]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 699960, 0, -10, 4700040]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 699960, 0, -10, 4700040]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32632', 'crs_transform': [20, 0, 699960, 0, -20, 4700040

Map(center=[49.8175, 15.473], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [ ]:
for year in range(start_year + 1, end_year + 1):
    # Rename bands for NDVI, NDWI, and NBR difference images
    diff_images[f"ndvi_diff_{year}"] = diff_images[f"ndvi_diff_{year}"].rename([f"NDVI_{year}"])
    diff_images[f"ndwi_diff_{year}"] = diff_images[f"ndwi_diff_{year}"].rename([f"NDWI_{year}"])
    diff_images[f"nbr_diff_{year}"] = diff_images[f"nbr_diff_{year}"].rename([f"NBR_{year}"])

for year in range(start_year + 1, end_year + 1):
    globals()[f"ndvi_diff_{year}"] = diff_images[f"ndvi_diff_{year}"]
    globals()[f"ndwi_diff_{year}"] = diff_images[f"ndwi_diff_{year}"]
    globals()[f"nbr_diff_{year}"] = diff_images[f"nbr_diff_{year}"]


In [ ]:
stats = ndwi_diff_2018.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=GEOMETRY,
    scale=3000
)
print(stats.getInfo())


{'NDWI_2018_max': 0.31167422235012054, 'NDWI_2018_min': -0.35610102117061615}


In [ ]:
# Parameters
CLD_PRB_THRESH = 10
GEOMETRY = ee.FeatureCollection('users/pwashaya9/czech_shp').geometry()

# # Debugging Mask Function
# def debug_add_cld_shdw_mask(img):
#     is_cloud = img.select('MSK_CLDPRB') #.gt(CLD_PRB_THRESH) \
#                   # .And(img.select('MSK_CLASSI_OPAQUE').gte(1)) \
#                   # .Or(img.select('MSK_CLASSI_CIRRUS').gte(1))
#     scl = img.select('SCL')
#     scl_mask = scl.eq(4).Or(scl.eq(8)).Or(scl.eq(9)).Or(scl.eq(10)).Or(scl.eq(11))

#     cloud_shadow_mask = is_cloud.And(scl_mask).rename('cloudmask')
#     return img.addBands(cloud_shadow_mask).updateMask(cloud_shadow_mask.Not())


def mask_clouds(image):
    """
    Masks clouds in Sentinel-2 TOA imagery using the QA60 band.

    Args:
        image (ee.Image): Sentinel-2 TOA image.

    Returns:
        ee.Image: The cloud-masked image.
    """
    # Get the QA60 band
    qa60 = image.select('QA60')

    # Create a mask for clear pixels (non-cloudy areas have a value of 0 in QA60)
    cloud_mask = qa60.bitwiseAnd(1 << 10).eq(0).And(qa60.bitwiseAnd(1 << 11).eq(0))

    # Apply the mask to the image
    return image.updateMask(cloud_mask)


# Visualize a Sample Year
sample_year = 2018
start_date = f"{sample_year}-05-01"
end_date = f"{sample_year}-09-30"

sample_collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                     .filterBounds(GEOMETRY)
                     .filterDate(start_date, end_date)
                    #  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                     .map(mask_clouds))

sample_image = sample_collection.median()

# Visualize
Map = geemap.Map(center=[49.8175, 15.4730], zoom=6)
Map.addLayer(sample_image.select(['B4', 'B3', 'B2']), {'min': 0, 'max': 3000}, 'Original Image')
# Map.addLayer(sample_image.select('cloudmask'), {'min': 0, 'max': 1, 'palette': ['black', 'red']}, 'Cloud Mask')
# Map.addLayer(sample_image.updateMask(sample_image.select('cloudmask').Not()),
#              {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
#              'Masked Image')
Map


KeyboardInterrupt: 

In [ ]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(GEOMETRY).filterDate('2017-05-01', '2017-07-31')
print(collection.first().bandNames().getInfo())


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']


In [ ]:
band_names = nbr_diff_2018.bandNames()
print(band_names.getInfo())

['NBR_2018']


In [ ]:
# Define the start and end years
start_year = 2016
end_year = 2024

# Define the GEOMETRY and Sentinel-1 collection
GEOMETRY = ee.FeatureCollection('users/pwashaya9/czech_shp').geometry()

def process_sar(year, polarization, orbit):
    """Process Sentinel-1 data for a specific year, polarization, and orbit."""
    start_date = f"{year}-08-01"
    end_date = f"{year}-09-30"

    collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                  .filterBounds(GEOMETRY)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
                  .filter(ee.Filter.eq('instrumentMode', 'IW'))
                  .filter(ee.Filter.eq('orbitProperties_pass', orbit))
                  .map(landuse))

    # Return the median composite
    return collection.select(polarization).mean()

# Create dictionaries to store results
sar_mosaics = {}
sar_differences = {}

# Process data for each year, polarization, and orbit
polarizations = ['VV', 'VH']
orbits = ['ASCENDING', 'DESCENDING']

for year in range(start_year, end_year + 1):
    sar_mosaics[year] = {}
    for polarization in polarizations:
        sar_mosaics[year][polarization] = {}
        for orbit in orbits:
            sar_mosaics[year][polarization][orbit] = process_sar(year, polarization, orbit)

# Compute differences for each polarization and orbit between consecutive years
for year in range(start_year + 1, end_year + 1):
    prev_year = year - 1
    sar_differences[year] = {}
    for polarization in polarizations:
        sar_differences[year][polarization] = {}
        for orbit in orbits:
            sar_differences[year][polarization][orbit] = (
                sar_mosaics[year][polarization][orbit]
                .subtract(sar_mosaics[prev_year][polarization][orbit])
            )

# Example: Accessing a specific difference image
example_diff = sar_differences[2023]['VH']['ASCENDING']

# Visualization parameters for SAR differences
vis_params = {'min': -20, 'max': 20, 'palette': ['blue', 'white', 'red']}

# Display a specific SAR difference image (e.g., VH, ascending, 2023)
import geemap
m = geemap.Map()
m.add_layer(example_diff, vis_params, 'SAR VH Ascending 2023 Diff')
m.centerObject(GEOMETRY, 8)
m

Map(center=[49.74359830110612, 15.33857109123141], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# Rename bands for SAR difference images
for year in range(start_year + 1, end_year + 1):
    for polarization in polarizations:
        for orbit in orbits:
            # Generate key for the difference image
            diff_key = sar_differences[year][polarization][orbit]

            # Rename the band for the SAR difference image
            renamed_band_name = f"{polarization}_{orbit}_{year}"
            sar_differences[year][polarization][orbit] = diff_key.rename([renamed_band_name])

            # Dynamically assign to a global variable
            globals()[f"{polarization.lower()}_{orbit.lower()}_diff_{year}"] = sar_differences[year][polarization][orbit]

In [ ]:
band_names = vh_ascending_diff_2023.bandNames()
print(band_names.getInfo())

['VH_ASCENDING_2023']


In [ ]:
# Where to save the trained model.
MODEL_DIR_SAR = 'gs://' + OUTPUT_BUCKET + '/logistic_sar_model'
# Where to save the EEified model.
EEIFIED_DIR_SAR = 'gs://' + OUTPUT_BUCKET + '/logistic_sar_eeified'

# Where to save the trained model.
MODEL_DIR_OPTICAL = 'gs://' + OUTPUT_BUCKET + '/logistic_optical_model'
# Where to save the EEified model.
EEIFIED_DIR_OPTICAL = 'gs://' + OUTPUT_BUCKET + '/logistic_optical_eeified'

In [ ]:
!pip install --upgrade tensorflow_addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import tensorflow_addons as tfa

from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR_OPTICAL, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR_OPTICAL} --dest_dir {EEIFIED_DIR_OPTICAL} --input {input_dict} --output {output_dict}

'{"serving_default_input_5:0": "array"}'
'{"StatefulPartitionedCall:0": "output"}'
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Successfully saved project id
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlo

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR_SAR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR_SAR} --dest_dir {EEIFIED_DIR_SAR} --input {input_dict} --output {output_dict}

'{"serving_default_input_5:0": "array"}'
'{"StatefulPartitionedCall:0": "output"}'
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Successfully saved project id
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlo

In [ ]:
REGION = 'global'


# # Name of the AI Platform model to be hosted.
# MODEL_NAME = 'saved_model_latest_testOPTICAL_SAR_10'
# # Version of the AI Platform model to be hosted.
# VERSION_NAME = 'v015_OPTICAL_SAR_10'

# For Optical Model
MODEL_NAME_OPTICAL = 'saved_model_optical'
VERSION_NAME_OPTICAL = 'v1_optical'

# For SAR Model
MODEL_NAME_SAR = 'saved_model_sar2'
VERSION_NAME_SAR = 'v1_sar2'

In [ ]:
!gcloud ai-platform models create {MODEL_NAME_OPTICAL} \
    --project {PROJECT} \
    --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME_OPTICAL} \
    --project {PROJECT} \
    --region {REGION} \
    --model {MODEL_NAME_OPTICAL} \
    --origin {EEIFIED_DIR_OPTICAL} \
    --framework "TENSORFLOW" \
    --runtime-version=2.11 \
    --python-version=3.7


!gcloud ai-platform models create {MODEL_NAME_SAR} \
    --project {PROJECT} \
    --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME_SAR} \
    --project {PROJECT} \
    --region {REGION} \
    --model {MODEL_NAME_SAR} \
    --origin {EEIFIED_DIR_SAR} \
    --framework "TENSORFLOW" \
    --runtime-version=2.11 \
    --python-version=3.7


Learn more about regional endpoints and see a list of available regions: https://cloud.google.com/ai-platform/prediction/docs/regional-endpoints
Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [awesome-treat-402108] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name

Learn more about regional endpoints and see a list of available regions: https://cloud.google.com/ai-platform/prediction/docs/regional-en

In [ ]:
sar_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME_SAR,
    version=VERSION_NAME_SAR,
    # Can be anything, but don't make it too big.
    inputTileSize=[4, 4],
    inputShapes={'array':[4]},
    # Keep this the same as your training data.
    proj=ee.Projection('EPSG:4326').atScale(10),

    fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

optical_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME_OPTICAL,
    version=VERSION_NAME_OPTICAL,
    # Can be anything, but don't make it too big.
    inputTileSize=[4, 4],
    inputShapes={'array':[3]},
    # Keep this the same as your training data.
    proj=ee.Projection('EPSG:4326').atScale(10),

    fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

In [ ]:
# Reference = Reference_training.rename('Band_1')
# BANDS =['ASCVV','ASCVH', 'DESCVV', 'DESCVH']
stack_2022 = vh_ascending_diff_2017.addBands(vv_ascending_diff_2017).addBands(vh_descending_diff_2017).addBands(vv_descending_diff_2017).addBands(nbr_diff_2017).addBands(ndwi_diff_2017).addBands(ndvi_diff_2017)







#BANDS =['ASCVV','ASCVH', 'DESCVV', 'DESCVH','NBR', 'NDWI', 'NDVI']

OPTICAL_BANDS = ['NBR_2017', 'NDWI_2017', 'NDVI_2017']  # Example

SAR_BANDS = ['VH_ASCENDING_2017','VV_ASCENDING_2017', 'VH_DESCENDING_2017', 'VV_DESCENDING_2017']  # Example

In [ ]:
# Define valid SAR and optical areas
sar_valid = stack_2022.select(SAR_BANDS).mask().reduce(ee.Reducer.allNonZero())
optical_valid = stack_2022.select(OPTICAL_BANDS).mask().reduce(ee.Reducer.allNonZero())

# Combine SAR and optical valid areas for complete coverage
combined_valid = sar_valid.Or(optical_valid)

# Prepare inputs for optical and SAR models
optical_input = stack_2022.select(OPTICAL_BANDS).toFloat().toArray()
sar_input = stack_2022.select(SAR_BANDS).toFloat().toArray()

# Predict using the optical model in valid optical areas
optical_predictions = (
    optical_model.predictImage(optical_input).arrayGet([0]).updateMask(optical_valid)
)

# Predict using the SAR model in valid SAR areas
sar_predictions = (
    sar_model.predictImage(sar_input).arrayGet([0]).updateMask(sar_valid)
)

# Combine predictions: optical where available, fallback to SAR otherwise
combined_predictions = (
    optical_predictions.unmask()  # Unmask ensures no gaps
    .where(optical_predictions.mask().Not(), sar_predictions)  # Use SAR where optical is missing
).updateMask(combined_valid)  # Restrict to valid combined areas

# Apply a classification threshold for binary predictions
combined_predictions = combined_predictions.gt(0.8).selfMask()

# Debugging: Visualization parameters
sar_valid_vis = {'min': 0, 'max': 1, 'palette': ['blue']}
optical_valid_vis = {'min': 0, 'max': 1, 'palette': ['green']}
combined_predictions_vis = {'min': 0, 'max': 1, 'palette': ['red', 'green']}
ndvi_vis = {'bands': ['NDVI_2017'], 'min': -1, 'max': 1}

# Fetch map tile layers for visualization
ndvi_mapid = stack_2022.select('NDVI_2017').getMapId(ndvi_vis)
sar_valid_mapid = sar_valid.getMapId(sar_valid_vis)
optical_valid_mapid = optical_valid.getMapId(optical_valid_vis)
combined_predictions_mapid = combined_predictions.getMapId(combined_predictions_vis)

# Set up the Folium map
map_center = [49.819663, 15.300458]
map = folium.Map(location=map_center, zoom_start=11)

# Add NDVI layer for reference
folium.TileLayer(
    tiles=ndvi_mapid['tile_fetcher'].url_format,
    attr='NDVI_2022',
    name='NDVI_2022',
    overlay=True
).add_to(map)

# Add SAR valid areas layer
folium.TileLayer(
    tiles=sar_valid_mapid['tile_fetcher'].url_format,
    attr='SAR Valid Areas',
    name='SAR Valid Areas',
    overlay=True
).add_to(map)

# Add Optical valid areas layer
folium.TileLayer(
    tiles=optical_valid_mapid['tile_fetcher'].url_format,
    attr='Optical Valid Areas',
    name='Optical Valid Areas',
    overlay=True
).add_to(map)

# Add Combined Predictions layer
folium.TileLayer(
    tiles=combined_predictions_mapid['tile_fetcher'].url_format,
    attr='Combined Predictions',
    name='Combined Predictions',
    overlay=True
).add_to(map)

# Add layer control and display the map
map.add_child(folium.LayerControl())

# Display the map
map


In [ ]:
forest_polygons = forest_mask.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,  # Adjust the scale based on your resolution needs
    maxPixels=1e13
)

In [ ]:
import time
# Define the export parameters
export_params = {
    'image': combined_predictions,
    'description': 'Predictions_combined_2017_cloudless',
    'folder': 'Tezba_predictions',  # Optional: specify a folder in your Drive
    'fileFormat': 'GeoTIFF',
    'region': GEOMETRY.getInfo()['coordinates'],
    'scale': 10,
    'maxPixels': 1e13
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Monitor the task
while export_task.status()['state'] == 'RUNNING':
    print('Exporting predictions...')
    time.sleep(10)
print('Exported predictions to Google Drive.')


Exported predictions to Google Drive.


In [ ]:


# Define function to export to Google Earth Engine Asset
def export_to_asset(image, asset_id, description):
    export_config = {
        'image': image,
        'description': description,
        'scale': 10,  # Sentinel-2 has a resolution of 10m for bands B2, B3, B4, and B8
        'region': GEOMETRY.getInfo()['coordinates'],
        'assetId': asset_id,
        'maxPixels': 1e13
    }
    task = ee.batch.Export.image.toAsset(**export_config)
    task.start()

# Export images to Asset
export_to_asset(combined_predictions, 'projects/awesome-treat-402108/assets/combined_predictions', 'combined_predictions')
# export_to_asset(nbr_diff.select('NBR'), 'projects/awesome-treat-402108/assets/NDBRDIFF_2023_10m', 'NDBRDIFF_2023_10m')
# export_to_asset(ndvi_diff.select('NDVI'), 'projects/awesome-treat-402108/assets/NDVIDIFF_2023_10m', 'NDVIDIFF_2023_10m')

# Uncomment if you wish to export these as well
# export_to_asset(mosaic2020, 'users/your_username/mosaic_15', 'mosaic_15')
# export_to_asset(mosaic2021, 'users/your_username/mosaic_17', 'mosaic_17')
